***The example code and container image were developed by On-Prem Cloud team member Negin Sobhani***

# Utilizing CICD on the On-Premise Cloud

GitHub can be combined with tools deployed on the On-Premise cloud to deploy a full CICD workflow for your applications. In this tutorial for the sake of time we are going to cover some of these topics more quickly.

***Note:*** Docker, Podman, or a similar container managment tool is required on your local machine to follow along with the next commands. If you are not already familiar with container building and associated topics it's best to pay attention to the example, and go back afterward to try some of the concepts on your own. Here's a [link to Get Docker](https://docs.docker.com/get-docker/)

## Clone Web application repository

On a machine that has docker, or a similar container management tool, clone the following repository:

https://github.com/negin513/neon_dashboard.git

This already contains a Python Bokeh web application that can be run locally or can be built in to a container. The README.md file contains all the information you need in order to test run the application locally as well as run via Docker. We will cover the docker steps here.

## Build a Docker image

After cloning the repository make sure you are working in the directory that contains the file `Dockerfile`. If it's not present you may need to run `cd neon_dashboard`. Once you are in that directory we can build the Docker image.

***Note:*** The Dockerfile command specifies the URL to allow connections to. This is not applicable for this demonstration. Please replace `negin-neon-test.k8s.ucar.edu` with `localhost:5006` to run this locally. For a deployment with a valid URL, this would be that URL. A wildcard can also be used but it allows all connections and is not secure.

`docker build -t ncote/neon-app:2024-1-23 .`

It should all finish properly and we can test by navigating to [http://localhost:5006/neon_dashboard](http://localhost:5006/neon_dashboard)

We now want to update the websocket origin in the dockerfile to match what we want to deploy to on the K8s cluster and rebuild. 

## Push to internal Container Registry

The following example also uses a project in Harbor that has already been configured for my user name and allows me to do administrative tasks. 

Before we can push the container image to the local registry we have to login with our CIT credentials like so:

`docker login hub.k8s.ucar.edu -u ncote`

***Note:*** Access is currently given out to LDAP members of CISL only. We can add users on a request basis. Submit a ticket following the process we covered earlier to request access to Harbor. 

Once we are logged in we can tag the image we just built with the Harbor address and then push it with the following commands:

`docker tag ncote/neon-dashboard:2024-1-23 hub.k8s.ucar.edu/ncote/neon-dashboard:2024-1-23`

`docker push hub.k8s.ucar.edu/ncote/neon-dashboard:2024-1-23`

When that is finished our image has been added to Harbor and we can look at Vulnerability scan results and configure permissions if we like. The URL for the Harbor UI is [https://hub.k8s.ucar.edu/](https://hub.k8s.ucar.edu/)

My Harbor repository is public, I am in control of this in the Configuration settings, so anyone can pull these images without needing a login. 

## Create Helm Chart

There is a service deployed on the CISL On-Premise Cloud, named Argo CD, that can be configured to look for Helm charts in a git repository. Administrator assistance is required to connect your applications Helm chart with Argo CD initially. Once it is in place your application will update automatically whenever changes are made to the Helm chart hosted in the git repository. 

We include links to a few different example helm charts in the Documentation. They can be found at this [link to Helm Charts & Argo CD](https://ncar.github.io/cisl-cloud/how-to/K8s/Hosting/deploy.html#helm-charts-argo-cd). The NEON Dashboard example we have used is a standalone container so we will copy the Helm chart template used there right in to the CISL-CCPP-Tutorial repository we have open now. 

***NOTE:*** JupyterHub does not allow you to copy whole folders. We will create a folder named `web-app-helm` and copy the required files in to it. A `templates` directory inside `web-app-helm` is also required.  

### Update Chart.yaml

`Chart.yaml` is mostly used to define your applications name and the current version deployed. We will go ahead and update the name to be `ccpp-tutorial-demo` and the version will be 0.1.0 and the appVersion will follow the Docker image tag we used, todays date.

***Note:*** Avoid using the tag `:latest`. Since this value would never change, there won't be any changes to the `Chart.yaml` or `values.yaml` file to trigger a diff and sync. 

### Update values.yaml

The `values.yaml` file contains the information we really care about when trying to deploy our applications. We will go ahead and update the following values: `name:`, `group:`, `path:`, `fqdn:`, `secretName:`, `image:`, & `port:`. In depth details on these values are included in our Documentation page and the README in the repository.   

***Note:*** The FQDN used at this point in time must match the URL provided to the allow-websocket-origin command used in the Dockerfile or traffic will be blocked

## Contact Admin

The nest steps all require Administrator assistance. Please submit a ticket once you have reached this step, include your Git URL, and an administrator will contact you to deploy your application. We will walk through these steps now so you can see what it looks like behind the scenes. 

## Docs Example

As a demonstration I will make a small change to our documentation site and show how GitHub actions and workflows can be used to provide a full CICD experience. It builds a new Docker image, pushed it to Docker Hub, and updates the Helm chart. There are example workflows that can be used as a reference at this [link to GitHub Actions](https://ncar.github.io/cisl-cloud/how-to/github-actions.html)